<p><b><center><span style="font-size:26px;" </span> <span style="font-family=Times new roman;"</span> Understanding different Mean Encodings schemes & Implementing them in Python</center></b></p>

<p><center><span style="font-size:16px;"</span> By Abdul Muqtadir()</center></p>


# 1. Introduction
Mean Encoding
1. In this tutorial we are going to Mean encode "Embarked" column. Using four different encoding schemes.
    * K-Fold Validation
    * Leave one out 
    * Smoothing
    * Expanding mean
2. Then we are going to find corelation in between every encoding scheme and target.

In [1]:
# Importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

# Reading Dataset
Train_Data = pd.read_csv("../input/simple-dataset-for-easy-practice-for-beginners/BrandEDA.csv")

In [2]:
# About Dataset

#Train_Data.shape
#Train_Data.info()
Train_Data.dtypes
#Train_Data.nunique()
#Train_Data["Item ID"].unique()

#Train_Data.head()


Brand                    object
Item ID                   int64
Ram(Gb)                 float64
StoraGe(Gb)             float64
Price(Rs)                 int64
Processor speed(GHz)    float64
 Touch                   object
Color                    object
Weight                  float64
DisplAy size(Inch)      float64
dtype: object

In [3]:
#Drop multiple columns (Just to make dataset bit easier for demonstration)
#Train_Data.drop(Train_Data.iloc[:, 14:26], axis=1, inplace=True)

In [4]:
Train_Data.sort_values(['Item ID'],inplace=True)

In [5]:
Train_Data["Brand"].replace({"Apple":"0", "Dell":"1"}, inplace = True)

In [6]:
Train_Data = Train_Data.astype({"Brand": int})

In [7]:
Train_Data.head()
IID_GMean = Train_Data['Item ID'].mean()

In [8]:
Train_Data.dtypes

Brand                     int64
Item ID                   int64
Ram(Gb)                 float64
StoraGe(Gb)             float64
Price(Rs)                 int64
Processor speed(GHz)    float64
 Touch                   object
Color                    object
Weight                  float64
DisplAy size(Inch)      float64
dtype: object

In [9]:
Train_Data.groupby('Item ID')["Brand"].mean()

Item ID
4     0.0
6     1.0
8     1.0
15    1.0
26    0.0
32    0.0
Name: Brand, dtype: float64

# Mean Encoding with K-Fold Validation
***Steps***
1. **Import K-Fold with K=5 and shuffle = False**
2. **Split Train data in to Tr_d and Val_d**
3. **Iterate through K folds (If K=x This iteration will repeat x times)**
    * Train and Validation -(Assign)- train and validation index
    * Find Target mean for Train Folds groupedby(Categorical feature under operation)
    * New Feature in Validation Fold -(Assign)- Validation fold (Categorical feature under operation) .map(Target mean of Train folds)
    * Dataset(Index val) -(Assign)- Validation fold
4. **Fill null values with global mean in new feature of Dataset**

In [10]:
%%time
# YOUR CODE GOES HERE

# Calculate sum of the target values using all the objects.
target_sum = Train_Data.groupby('Item ID')['Brand'].transform('sum')

# Then subtract the target of the given object and divide the resulting value by n_objects - 1.
n_objects = Train_Data.groupby('Item ID')['Brand'].transform('count')

Train_Data['item_target_enc'] = (target_sum - Train_Data['Brand']) / (n_objects - 1)
Train_Data['item_target_enc'].fillna(0.3343, inplace=True)


CPU times: user 4.25 ms, sys: 44 µs, total: 4.3 ms
Wall time: 5.46 ms


In [11]:
Train_Data.shape
Train_Data.head(30)

,Brand,Item ID,Ram(Gb),StoraGe(Gb),Price(Rs),Processor speed(GHz),Touch,Color,Weight,DisplAy size(Inch),item_target_enc
9,0,4,16.0,NaN,382000,2.60,No,Silver,2.00,16.0,0.0
1,0,4,8.0,512.0,232000,3.20,No,Dark Black,1.29,13.3,0.0
3,0,4,8.0,NaN,195000,2.30,No,Space grey,1.37,13.3,0.0
4,0,4,4.0,128.0,92400,1.80,No,Silver,1.35,11.6,0.0
5,0,4,8.0,256.0,60000,2.60,No,Silver,1.00,13.3,0.0
7,0,4,8.0,128.0,224700,1.80,No,Silver,1.23,13.3,0.0
23,0,4,8.0,256.0,255000,2.30,No,Silver,1.37,13.3,0.0
16,1,6,8.0,256.0,185000,1.80,No,Silver,1.23,13.3,1.0
15,1,6,12.0,512.0,130000,2.10,Yes,Silver,1.95,15.6,1.0
28,1,6,6.0,650.0,33200,2.26,No,White,2.84,15.6,1.0


In [12]:
%%time

# YOUR CODE GOES HERE
alpha = 100

item_id_target_mean = Train_Data.groupby('Item ID')['Brand'].transform('mean')
n_objects = Train_Data.groupby('Item ID')['Brand'].transform('count')

Train_Data['item_target_enc'] = (item_id_target_mean * n_objects + 0.3343*alpha) / (n_objects + alpha)

Train_Data['item_target_enc'].fillna(0.3343, inplace=True)

CPU times: user 3.67 ms, sys: 128 µs, total: 3.79 ms
Wall time: 3.51 ms


In [13]:
Train_Data.shape
Train_Data.head()

,Brand,Item ID,Ram(Gb),StoraGe(Gb),Price(Rs),Processor speed(GHz),Touch,Color,Weight,DisplAy size(Inch),item_target_enc
9,0,4,16.0,NaN,382000,2.6,No,Silver,2.00,16.0,0.31243
1,0,4,8.0,512.0,232000,3.2,No,Dark Black,1.29,13.3,0.31243
3,0,4,8.0,NaN,195000,2.3,No,Space grey,1.37,13.3,0.31243
4,0,4,4.0,128.0,92400,1.8,No,Silver,1.35,11.6,0.31243
5,0,4,8.0,256.0,60000,2.6,No,Silver,1.00,13.3,0.31243
